# Đồ án cuối kỳ - Phân tích chủ đề văn bản
(Cập nhật 07/01/2021)

Nhóm: 12

Thành viên nhóm:
- Vũ Đăng Hoàng Long - MSSV: 18120203
- Nguyễn Huỳnh Đại Lợi - MSSV: 18120198

---
## Import thư viện

In [53]:
import requests
from bs4 import BeautifulSoup
import json
import time # Dùng để sleep chương trình
import pandas as pd
import re

---
## 1. Thu thập dữ liệu

Dữ liệu trong đồ án này được thu thập toàn bộ từ trang báo mạng Tuổi Trẻ Online (https://tuoitre.vn/).

Khi mới vào trang chủ, có thể thấy trang báo này phân bổ thứ tự và cấu trúc thông tin không đều, rất khó để có thể thu thập được. Tuy nhiên, may mắn là trang báo này có mục tin mới nhất chứa toàn bộ tin từ các chuyên mục và có tổ chức các mục có cấu trúc rõ ràng (Hình 1).

![Hình 1](resources/tin-moi-nhat.png "Hình 1")

Khi người dùng kéo xuống cuối trang, sẽ có nút "Xem thêm" (Hình 2). Người dùng nhấn vào đây và tin tức cũ hơn sẽ hiện ngay bên dưới để người dùng kéo xuống tiếp.

![Hình 2](resources/xem-them.png "Hình 2")

Thoáng nhìn qua, quy trình lướt tin tức sẽ bao gồm: Vào mục tin mới -> Kéo xuống cuối trang -> Bấm "Xem thêm" -> Kéo xuống cuối trang và lặp lại. Như vậy với cách làm đơn giản nhất là mô phỏng click chuột của người dùng để bấm nút xem thêm càng nhiều càng tốt, sau đó lấy file HTML của trang về và thực hiện parse (Đây chính là cách làm trong phiên bản đầu của quy trình này của nhóm). Tuy nhiên cách này rất thiếu hiệu quả bởi lẽ máy sẽ cần phải mở trình duyệt lên và mô phỏng thao tác của người dùng -> rất chậm do thời gian tải trang và tốn tài nguyên xử lý của máy tính do phải render trang web. Trong phiên bản đầu nhóm đã mất khoảng vài phút để có thể lấy được 480 tin.

Nhận thấy rằng trang tin này có một đặc điểm là có thể lăn chuột vô tận, nhóm tìm hiểu và phát hiện rằng có thể cải thiện hiệu suất của quy trình này lên rất nhiều lần! Cụ thể, khi người dùng bấm vào nút "Xem thêm", trang web sẽ gửi yêu cầu lên server và nhận về một file HTML có cấu trúc đơn giản chứa thông tin các bài báo (Hình 3). Sau đó trình duyệt sẽ thực hiện thay đổi DOM của trang web để thêm các tin này vào bên dưới, tạo hiệu ứng lăn chuột vô tận. Do vậy, thay vì phải đợi trình duyệt render trang web, nhóm có thể request thẳng đường link giống cách trang web request lên server. Việc làm này sẽ rút ngắn rất nhiều thời gian của quá trình thu thập dữ liệu!

![Hình 3](resources/xem-them-network.png "Hình 3")

Khi sử dụng công cụ theo dõi network của trình duyệt, nhóm phát hiện ra rằng đường link request tin mới của trang web khi bấm nút xem thêm lần đầu là "https://tuoitre.vn/timeline/0/trang-1.htm". Rất có thể khi thay thế trang-1 thành trang-2, trang-3,... chúng ta sẽ thu được các kết quả là tin mới của các lần bấm thứ 2, thứ 3,... vào nút "Xem thêm" (điều này đúng, nhóm đã kiểm chứng).

Đoạn code bên dưới sẽ thực hiện request vào đường link trên và thực hiện parse với BeautifulSoup như HTML bình thường để lấy đường link và chuyên mục của các tin tức. Sau đó sẽ thực hiện request trang tin chính từ đường link thu thập được và lấy các thông tin tiêu đề, mô tả và nội dung.

Kết quả sẽ được in bằng hàm tail() giúp có thể nắm được số lượng data lấy được thông qua index.

In [56]:
# Thử nghiệm url đầu tiên
url = "https://tuoitre.vn/timeline/0/trang-1.htm"

# Lấy danh sách news_items
html_text = requests.get(url).text
html_tree = BeautifulSoup(html_text, 'html.parser')
news_items = html_tree.findAll('li', class_='news-item')

# Lấy ra link, title và category từ news_items
raw_data = pd.DataFrame(columns=["links","title","description","content","class"])
for item in news_items:
    title = item.find('h3', class_='title-news').text.replace('\n','')
    link = "https://tuoitre.vn" + item.find('a').attrs["href"]
    category = item.find('a', class_='category-name').text
    raw_data = raw_data.append({"links":link, "title":title, "class":category}, ignore_index=True)

raw_data.tail()

,links,title,description,content,class
15,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",NaN,NaN,Sức khỏe
16,https://tuoitre.vn/lang-kinh-24g-canh-giac-voi...,Lăng kính 24g: Cảnh giác với tội phạm trộm cắp...,NaN,NaN,Cần biết
17,https://tuoitre.vn/game-show-cuoi-cung-chi-tai...,Gameshow cuối cùng của Chí Tài - Cơ hội đổi đờ...,NaN,NaN,Giải trí
18,https://tuoitre.vn/giua-kho-khan-thoi-covid-19...,"Giữa khó khăn 'thời COVID-19', một doanh nghiệ...",NaN,NaN,Thời sự
19,https://tuoitre.vn/triet-pha-duong-day-mang-th...,"Triệt phá đường dây mang thai hộ, mỗi lần 'đẻ ...",NaN,NaN,Pháp luật


In [57]:
%%time
for _, row in raw_data.iterrows():
    news_page = requests.get(row["links"]).content
    news_tree = BeautifulSoup(news_page, "html.parser")
    # Lấy mô tả
    try:
        row["description"] = news_tree.find("h2", class_="sapo").text
    except:
        row["description"] = ''
    # Lấy nội dung
    try:
        body = news_tree.find("div", id="main-detail-body")
        content = body.findChildren("p", recursive=False)
        row["content"] = ""
        for x in content:
            row["content"] += x.text
    except:
        row["content"] = ''
    time.sleep(0.05)
    
raw_data.tail()

Wall time: 2.83 s


,links,title,description,content,class
15,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",TTO - Sau nhiều ngày số mắc mới khá cao và dồn...,"Theo Bộ Y tế, 1 ca mắc mới ngày 6-1 là ca nhập...",Sức khỏe
16,https://tuoitre.vn/lang-kinh-24g-canh-giac-voi...,Lăng kính 24g: Cảnh giác với tội phạm trộm cắp...,,,Cần biết
17,https://tuoitre.vn/game-show-cuoi-cung-chi-tai...,Gameshow cuối cùng của Chí Tài - Cơ hội đổi đờ...,"TTO - Tối 5-1, tập đầu tiên của chương trình t...",Cơ hội đổi đời là chương trình truyền hình thự...,Giải trí
18,https://tuoitre.vn/giua-kho-khan-thoi-covid-19...,"Giữa khó khăn 'thời COVID-19', một doanh nghiệ...",TTO - Hầu hết doanh nghiệp tại Đồng Nai duy tr...,"Ngày 6-1, Sở Lao động - thương binh và xã hội ...",Thời sự
19,https://tuoitre.vn/triet-pha-duong-day-mang-th...,"Triệt phá đường dây mang thai hộ, mỗi lần 'đẻ ...",TTO - Công an quận Long Biên (Hà Nội) vừa triệ...,"Ngày 6-1, Công an quận Long Biên (Hà Nội) cho ...",Pháp luật


### Vấn đề phát sinh trong quá trình thu thập dữ liệu tự động hàng loạt

#### Thiếu dữ liệu làm đứt quãng quá trình thu thập
Do khối lượng dữ liệu có thể thu thập được rất lớn (ước tính lên đến hơn 500.000 tin), việc giám sát quá trình khai thác liên tục là không thể. Việc xuất hiện lỗi này đã khiến quá trình khai thác bị ngắt quãng mỗi khi xuất hiện. Nhóm đã khắc phục bằng cách đặt các khối lệnh ```try``` ```catch``` ở mỗi đoạn code parse các phần thông tin, tự động trả về rỗng nếu có lỗi.

#### Lỗi TooManyRedirects: Exceeded 30 redirects
Đây là lỗi xuất hiện khi một trang web tự điều hướng quá 30 lần (con số mặc định có thể chỉnh được).

Với một số server, khi được request sẽ tùy vào thông tin header của request (như trình duyệt, cache,...), server sẽ đính kèm lệnh chuyển hướng vào response và trình duyệt sẽ dùng thông tin đó điều hướng chuyển qua trang web nhất định. Chẳng hạn, khi người dùng điện thoại truy cập vào trang facebook.com, trình duyệt sẽ tự động điều hướng chuyển người dùng tới trang m.facebook.com là trang có giao diện dành cho điện thoại. Request HTTP trong python cũng không phải ngoại lệ khi có thiết lập mặc định là sẽ tự động chuyển hướng cho đến khi không còn nhận được lệnh từ server.

Khi tra cứu lỗi này trên google, nhóm nhận được các đề xuất giải quyết sau:
- Thêm tham số ```allow_redirects=True``` vào lệnh ```requests.get```
- Thêm tham số ```headers``` vào lệnh ```requests.get```. Tham số này chứa các thông tin mặc định của header mà trình duyệt gửi tới server.
- Tạo một ```session``` và thay đổi thuộc tính ```max_redirect``` lên hớn 30.

Với cách 1 vô nghĩa do tham số đó là mặc định sẵn, cách 3 thì không đảm bảo thay đổi bao nhiêu là đủ. Còn với cách 2, lệnh request cung cấp thêm thông tin cho server nhằm tránh tình trạng server không xác định được người dùng và rơi vào vòng lặp redirect vô tận. Tuy nhiên nhóm áp dụng và vẫn không khắc phục được. Khi thử lấy link này thì hoàn toàn không vô được.

Một điều thú vị là khi debug, nhóm phát hiện rằng các link sinh ra lỗi này đều có nội dung về giáo dục (Hình 4). Khi thử tìm kiếm trang riêng có đường link https://tuyensinh.tuoitre.vn/ thì được điều hướng trở về trang https://tuoitre.vn/giao-duc/tuyen-sinh.htm. Kết luận của nhóm: rất có thể trang báo này đã có quá trình thay đổi công nghệ ở một thời điểm nhất định trong quá khứ khiến cho các trang này không truy cập được -> không chỉ trang tin giáo dục mà còn có thể nhiều trang tin khác cũng bị tương tự.

![Hình 4](resources/tuyen-sinh-errors.png "Hình 4")

Quyết định của nhóm là sẽ áp dụng cách khắc phục 2 vì nó hợp lý và áp khối ```try``` ```catch``` điền thông tin trống với các link tiếp tục bị lỗi.

#### Lỗi 502 của trang web được request về
Lỗi này đến từ phía server của trang báo và xuất hiện rất ngẫu nhiên không xác định được. Nhóm đã giải quyết bằng cách đặt vòng lặp ```while``` tại các câu lệnh ```get``` và chỉ ngừng khi quá số lần cho phép hoặc nhận phản hồi 200 tức OK. Tuy nhiên cần chú ý cho phép cả phản hồi 301 vì đây là lệnh điều hướng trang có liên quan tới lỗi ở phần trước có đề cập.

### Thiết lập đoạn code thu thập dữ liệu tự động hàng loạt

In [59]:
def single_request_scraping(index = 1, sleep_time = 0.05):
    '''
    Thu thập các trang tin trong mục tin mới nhất của báo Tuổi Trẻ.
    Mỗi lần thu thập 20 tin. Sử dụng số index để thu thập các trang liên tiếp.
    Với index = 1 tức lấy 20 trang tin mới nhất.

    Lưu ý:
        Khi lấy liên tục cần chọn thời điểm
        vì có khả năng báo cập nhật tin tức mới sẽ làm 
        tin ở trang trước bị đẩy xuống trang sau.

    Param:
        index: số chỉ trang cần request.
    '''
    allow_status = [200, 301]
    s = requests.Session()
    s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    url = f"https://tuoitre.vn/timeline/0/trang-{index}.htm"

    # Lấy danh sách news_items
    try:
        response = s.get(url)
        try_left = limit_retry
        while (response.status_code not in allow_status and try_left > 0):
            print(f"Loi {response.status_code} tai trang {index}")
            response = s.get(url)
            try_left -= 1
        html_text = response.text
    except:
        print(f"Loi request tai trang {index}")
        return None
    
    html_tree = BeautifulSoup(html_text, 'html.parser')
    news_items = html_tree.findAll('li', class_='news-item')
    
    # Kiểm tra lỗi nếu không lấy được bất cứ item nào
    if (len(news_items) == 0):
        print(f'Trang {index} khong lay duoc item.')
        print(html_text)
        return None
    elif (len(news_items) != 20):
        print(f"Warning: Trang {index} chi lay duoc {len(news_items)} items.")
    
    # Lấy ra link, title và category từ news_items
    raw_data = pd.DataFrame(columns=["links","title","description","content","class"])
    for item in news_items:
        try:
            title = item.find('h3', class_='title-news').text.replace('\n','')
        except:
            title = ""
        link = "https://tuoitre.vn" + item.find('a').attrs["href"]
        try:
            category = item.find('a', class_='category-name').text
        except:
            category = ""
        raw_data = raw_data.append({"links":link, "title":title, "class":category}, ignore_index=True)
    
    # Tiến hành lấy nội dung từng link
    for _, row in raw_data.iterrows():
        try:
            response = s.get(row["links"])
            try_left = limit_retry
            while (response.status_code not in allow_status and try_left > 0):
                print(f"Loi {response.status_code} tai link {row['links']}")
                response = s.get(row["links"])
                try_left -= 1
            news_page = response.content
        except:
            print(f"Loi request tai link {row['links']}")
            row["description"] = ""
            row["content"] = ""
            continue
            
        news_tree = BeautifulSoup(news_page, "html.parser")
        # Lấy mô tả
        try:
            row["description"] = news_tree.find("h2", class_="sapo").text
        except:
            row["description"] = ''
        # Lấy nội dung
        try:
            body = news_tree.find("div", id="main-detail-body")
            content = body.findChildren("p", recursive=False)
            row["content"] = ""
            for x in content:
                row["content"] += x.text
        except:
            row["content"] = ''
        time.sleep(sleep_time)
    
    return raw_data

In [52]:
#Batch scraping
iter_num = 1  # Số batch bắt đầu
num = 200  # Số trang link trong 1 batch
continue_flag = True # Cờ hiệu kết thúc vòng lặp khi xảy ra lỗi

while (continue_flag):
    '''
    Khởi tạo dataframe rỗng. 
    Sau đó lấy đủ 1 số trang cho 1 batch.
    Rồi export file csv.
    '''
    batch_df = pd.DataFrame(columns=["links","title","description","content","class"])
    for index in range(iter_num*num+1,(iter_num+1)*num+1):
        data = single_request_scraping(index,0)
        if (data is None):
            continue_flag = False
            break
        print(f"Page {index} complete!")
        batch_df = batch_df.append(data)
    batch_df.to_csv(f'scraped_data\crawling_{iter_num}.csv',index=False,encoding="utf-8")    
    iter_num+=1

Page 3401 complete!
Page 3402 complete!
Page 3403 complete!
Page 3404 complete!
Page 3405 complete!
Page 3406 complete!
Page 3407 complete!
Page 3408 complete!
Page 3409 complete!
Page 3410 complete!
Page 3411 complete!
Page 3412 complete!
Page 3413 complete!
Page 3414 complete!
Page 3415 complete!
Page 3416 complete!
Page 3417 complete!
Page 3418 complete!
Page 3419 complete!
Page 3420 complete!
Page 3421 complete!
Page 3422 complete!
Page 3423 complete!
Page 3424 complete!
Page 3425 complete!
Page 3426 complete!
Page 3427 complete!
Page 3428 complete!
Page 3429 complete!
Page 3430 complete!
Page 3431 complete!
Page 3432 complete!
Page 3433 complete!
Page 3434 complete!
Page 3435 complete!
Page 3436 complete!
Page 3437 complete!
Page 3438 complete!
Page 3439 complete!
Page 3440 complete!
Page 3441 complete!
Page 3442 complete!
Page 3443 complete!
Page 3444 complete!
Page 3445 complete!
Page 3446 complete!
Page 3447 complete!
Page 3448 complete!
Page 3449 complete!
Page 3450 complete!


FileNotFoundError: [Errno 2] No such file or directory: 'scraped_data\\crawling_17.csv'

## 2. Khám phá dữ liệu